In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import higlass
import higlass.client as hgc
import higlass.tilesets as hti

ts1 = hti.cooler('../data/Dixon2012-J1-NcoI-R1-filtered.100kb.multires.cool')

tr1 = hgc.Track('top-axis')
tr2 = hgc.Track('heatmap', tileset=ts1, height=100)
higlass.display([hgc.View([tr1, tr2])])()

terminating: NljKVZvFR0qte9_YCgyxJw
self.tilesets: [<higlass.tilesets.Tileset object at 0x10d4a3f98>]
 * Serving Flask app "higlass.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'WGbe-4iVRUeuN2S_NwlhuA', 'tracks': {'top': [{'ty…

In [15]:
import h5py
import clodius.tiles.mrmatrix as hgmm
import clodius.tiles.format as hgfo
import clodius.tiles.utils as hgut
import higlass.tilesets as hfti

f = h5py.File('../data/int_matrices.hdf5', 'r')
t = hgmm.tiles(f, 0, 0, 0)

ts = hfti.Tileset(
    tileset_info=lambda: hgmm.tileset_info(f),
    tiles=lambda tile_ids: hgut.tiles_wrapper_2d(tile_ids,
                    lambda z,x,y: hgfo.format_dense_tile(
                        hgmm.tiles(f, z, x, y)))
    )

tr1 = hgc.Track('top-axis')
tr2 = hgc.Track('heatmap', tileset=ts, height=200)
higlass.display([hgc.View([tr1, tr2])])()

attrs: ['max-zoom', 'min-pos', 'max-pos']
terminating: Z7dx4y0TRxWRcKgGk9JGiw
self.tilesets: [<higlass.tilesets.Tileset object at 0x10d4a31d0>]
 * Serving Flask app "higlass.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'HZ8RP_XgQs-cLkHnDzgH7w', 'tracks': {'top': [{'ty…

attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'max-pos']
attrs: ['max-zoom', 'min-pos', 'ma